# Manipulating Datasets

> **Set up**
>
> To run this notebook, first install the Julia kernel for Jupyter Notebooks using [IJulia](https://julialang.github.io/IJulia.jl/stable/manual/installation/), then [create an environment](https://pkgdocs.julialang.org/v1/environments/) for this tutorial with the packages listed with `using <PackageName>` further down.
>
> This tutorial has demonstrated compatibility with these package versions. If you run into any errors, first check your package versions for consistency using `Pkg.status()`.
>
 > ```
 > Status `~/work/PowerSystems.jl/PowerSystems.jl/docs/Project.toml`
 >   [336ed68f] CSV v0.10.15
 >   [a93c6f00] DataFrames v1.8.1
 >   [864edb3b] DataStructures v0.19.3
 > ⌅ [e30172f5] Documenter v1.15.0
 >   [d12716ef] DocumenterInterLinks v1.1.0
 >   [a078cd44] DocumenterMermaid v0.2.0
 >   [2cd47ed4] InfrastructureSystems v3.3.0
 >   [b6b21f68] Ipopt v1.14.0
 >   [0f8b85d8] JSON3 v1.14.3
 >   [4076af6c] JuMP v1.29.4
 >   [98b081ad] Literate v2.21.0
 >   [f00506e0] PowerSystemCaseBuilder v2.2.0
 >   [bcd98974] PowerSystems v5.5.0 `~/work/PowerSystems.jl/PowerSystems.jl`
 >   [08abe8d2] PrettyTables v3.2.1
 >   [9e3dc215] TimeSeries v0.25.2
 >   [04da0e3b] TypeTree v0.3.0
 >   [ade2ca70] Dates v1.11.0
 > Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`
 > 
 > ```

`PowerSystems` provides function interfaces to all data, and in this tutorial we will explore how to do this using the `show_components`,
`get_component` where {T <: Component})/
`get_components`, and getter (`get_*`) and setter (`set_*`) functions for component fields.

## Viewing Components in the System
We are going to begin by loading in a test case `System` from [`PowerSystemCaseBuilder.jl`](https://nrel-sienna.github.io/PowerSystemCaseBuilder.jl/stable/):

In [ ]:
using PowerSystems;
using PowerSystemCaseBuilder;
sys = build_system(PSISystems, "c_sys5_pjm")

Notice that the print statement for the `System` already includes a basic
summary of the components, including 5 `ThermalStandard` components.
We can use the `show_components` function to get more details:

In [ ]:
show_components(ThermalStandard, sys)

We can see the names and availability are the standard fields returned when using `show_components`.
We can also view specific fields within components using the `show_components`
function. For example, we can view the type of `fuel` the thermal generators are using,
and their current `active_power` and `reactive_power` for a power flow case:

In [ ]:
show_components(ThermalStandard, sys, [:fuel, :active_power, :reactive_power])

Notice all our thermal generators are currently fueled by coal.

## Accessing and Updating a Component in a System
We can access a component in our system using the
`get_component` where {T <: Component})
function. For example, if we are interested in accessing a `ThermalStandard` component we can
do so using the component's name and `Type` from `PowerSystems.jl`'s Type Tree.
From above we know the names of the thermal generators.

In [ ]:
solitude = get_component(ThermalStandard, sys, "Solitude")

Notice that all of Solitude's fields are pretty-printed with the return statement for
quick reference. However, what is returned is a `ThermalStandard` object we can
manipulate:

In [ ]:
typeof(solitude)

If we are interested in accessing a particular field, we can use a `get_*` function, also known as a getter,
on this object. For example, if we are interested in the `fuel` we can use `get_fuel`):

In [ ]:
get_fuel(solitude)

You can see a `ThermalFuels` option returned.
To recap, `get_component` will return a component object, but we can use a specific `get_*` function to return the data in a particular field.
> *Warning*
>
> Using the "dot" access to get a field value from a component is actively discouraged, use `get_*` functions instead.
> Julia syntax enables access to this data using the "dot" access (e.g., `solitude.fuel`), however this is discouraged for two reasons:
>  1. We make no guarantees on the stability of component structure definitions. We will maintain version stability on the accessor methods.
>  2. Per-unit conversions are made in the return of data from the accessor functions. (see the [per-unit](https://nrel-sienna.github.io/PowerSystems.jl/stable/explanation/per_unit/#per_unit) section for more details)
To update a field we can use a specific `set_*`, or setter function, which are defined for each component field.
We can use `set_fuel!`) to update the `fuel` field of Solitude to natural gas.

In [ ]:
set_fuel!(solitude, ThermalFuels.NATURAL_GAS)

We can use `show_components` again to check that the `Solitude` `fuel` has been
updated to `ThermalFuels.NATURAL_GAS`:

In [ ]:
show_components(ThermalStandard, sys, [:fuel])

Similarly, you can updated the `active_power` field using its specific `get_*` and `set_*` functions.
We can access this field by using `get_active_power`):

In [ ]:
get_active_power(solitude)

We can then update it using `set_active_power!`):

In [ ]:
set_active_power!(solitude, 4.0)

We can see that our `active_power` field has been updated to 4.0.

## Accessing and Updating Multiple Components in the System at Once
We can also update more than one component at a time using the `get_components` where {T <: Component}) and `set_*` functions.
Let's say we were interested in updating the `base_voltage` field for all of the `ACBus`.
We can see that currently the `base_voltages` are:

In [ ]:
show_components(ACBus, sys, [:base_voltage])

But what if we are looking to correct them to 250.0 kV?
Let's start by getting an iterator for all the buses using `get_components` where {T <: Component}):

In [ ]:
buses = get_components(ACBus, sys)

See that the pretty-print summarizes this set of components, but let's check what was actually returned:

In [ ]:
typeof(buses)

> *Tip*
>
> Notice that `get_components` where {T <: Component}) and similar functions return Julia iterators, which allows you to
> access and manipulate data without a large memory allocation that might occur for very
> large data sets.
> Use [`collect`](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple%7BAny%7D)
> to gather the data to a vector instead, but be aware of your dataset size.
Now using the `set_base_voltage!` function and a `for` loop we can update the voltage:

In [ ]:
for i in buses
    set_base_voltage!(i, 250.0)
end

We could use `show_components` to verify the results, but this time let's use a
`get_*` function and Julia's dot notation over our bus iterator to get the data for
a specific field from multiple components:

In [ ]:
get_base_voltage.(buses)

We can see that all of the buses now have a `base_voltage` of 250.0 kV.
If we are interested in updating the `fuel` in all the thermal generators, we would use a similar approach. We begin by grabbing an iterator for all the components in `ThermalStandard`.

In [ ]:
thermal_gens = get_components(ThermalStandard, sys)

Now, using the `set_fuel!`) and a `for` loop, we will update the `fuel` to `NATURAL_GAS`.

In [ ]:
for i in thermal_gens
    set_fuel!(i, ThermalFuels.NATURAL_GAS)
end

We can verify that the `fuel` types for all the thermal generators has been updated,
using dot notation again to access the `fuel` fields:

In [ ]:
get_fuel.(get_components(ThermalStandard, sys))

See that we linked two functions here with Julia's dot notation -- this is a very convenient way of quickly getting the data you need.

## Filtering Specific Data
We have seen how to update a single component, and all the components of a specific type, but what if we are interested in updating only particular components? We can do this using filter functions.
For example, let's say we are interested in updating all the `active_power` values of the thermal generators except `Solitude`.
Let's start by seeing the current `active_power` values.

In [ ]:
show_components(ThermalStandard, sys, [:active_power])

Let's grab an iterator for the all the thermal generators except `Solitude` by adding a filter function
in another version of the `get_components` where {T <: Component}) function defined with Julia's multiple dispatch:

In [ ]:
thermal_not_solitude = get_components(x -> get_name(x) != "Solitude", ThermalStandard, sys)

We can see that only four `ThermalStandard` components are returned, as expected.
Now let's update the `active_power` field of these four thermal generators using the `set_active_power!` function.

In [ ]:
for i in thermal_not_solitude
    set_active_power!(i, 0.0)
end

Let's check the update using `show_components`:

In [ ]:
show_components(ThermalStandard, sys, [:active_power])

We can see that all the `active_power` values are 0.0, except `Solitude`.
We can filter on any component field. Similarly, let's filter all of the thermal generators
that now have an `active_power` of 0.0, and also set their availability to false.

In [ ]:
for i in get_components(x -> get_active_power(x) == 0.0, ThermalStandard, sys)
    set_available!(i, 0)
end

## Getting Available Components
The `get_available_components` function is a useful short-hand function with a
built-in filter for grabbing all the components of a particular type that are available.
For example, if we are interested in grabbing all the available `ThermalStandard`:

In [ ]:
get_available_components(ThermalStandard, sys)

We only retrieved one component, because we just set the rest to unavailable above:

In [ ]:
show_components(ThermalStandard, sys)

## Getting Buses
We can retrieve the `ACBus` components using
`get_buses`),
by ID number) or
`Area` or `LoadZone`).
Let's begin by accessing the ID numbers associated with the `ACBus`
components using the `get_bus_numbers` function.

In [ ]:
get_bus_numbers(sys)

We can see that these bus IDs are numbered 1 through 5.
Now let's specifically grab buses 2 and 3 using the `get_buses` function:

In [ ]:
high_voltage_buses = get_buses(sys, Set(2:3))

and update their base voltage to 330 kV:

In [ ]:
for i in high_voltage_buses
    set_base_voltage!(i, 330.0)
end

As usual, we can review the updated data with `show_components`:

In [ ]:
show_components(ACBus, sys, [:number, :base_voltage])

## Updating Component Names
We can also access and update the component name field using the
`get_name`) and
set_name!) functions.
Recall that we created an iterator called `thermal_gens` for all the thermal generators.
We can use the `get_name`) function to access
the `name` field for these components with dot notation:

In [ ]:
get_name.(thermal_gens)

To update the names we will use the
set_name!) function.
> *Warning*
>
> Specifically when using set_name!)
> to modify multiple components accessed through an iterator, it is important to note that this
> not only changes the field `name`, but also changes the iterator itself
> as you are iterating over it, which will result in unexpected outcomes and errors.
Therefore, rather than using set_name!)
on an iterator, only use it after first
calling [`collect`](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple%7BAny%7D)
on the iterator to get a vector of the components:

In [ ]:
for thermal_gen in collect(get_components(ThermalStandard, sys))
    set_name!(sys, thermal_gen, get_name(thermal_gen) * "-renamed")
end

Now we can check the names using the `get_name` function again.

In [ ]:
get_name.(get_components(ThermalStandard, sys))

Be aware again that accessing components through a vector using
[`collect`](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple%7BAny%7D)
might cause large memory allocations, based on your dataset size.

## Next Steps & Links
In this tutorial, we explored a dataset using `show_components` to summarize data
and accessed particular groups of components with `get_components` where {T <: Component}), `get_buses`),
and `get_available_components`.
We used specific `get_*` functions and `set_*` functions to see and update the fields in
`ThermalStandard` and `ACBus` components, but remember that these getters
and setters are available for each data field for components of all Types in `PowerSystems.jl`.
Follow the next tutorials to learn how to work with time series.